## Acoustic Integration - Automatic Differentiation

http://localhost:8888/?token=sloth

In [ ]:
# from tqdm import trange
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib ipympl

import drjit as dr
import mitsuba as mi

from libs import utils, acoustic_torch

mi.set_variant('cuda_ad_acoustic')
mi.set_log_level(mi.LogLevel.Warn)

plt.style.use('ggplot')
utils.drjit_turn_off_optimizations(False)

sess_seed   = np.random.randint(0, 2**30)
sess_seed_g = np.random.randint(0, 2**30)
print(f"session seeds are: sess_seed={sess_seed}; sess_seed_g={sess_seed_g}")

### Scene Construction

In [ ]:
absorption = [0.1, 0.75]
scattering = [0.5, 0.75]

absorption_mdim, scattering_mdim = np.meshgrid(absorption, scattering)
absorption,      scattering      = absorption_mdim.flatten(), scattering_mdim.flatten()

config = {
    "box_dim":     [25., 12., 7.],
    "mic_pos":     [ 9.,  6., 1.],
    "speaker_pos": [20.,  7., 2.],
    "speaker_radius": 0.5, #0.1,

    "absorption": [(i + 1, a) for i, a in enumerate(absorption)],
    "scattering": [(i + 1, s) for i, s in enumerate(scattering)],

    "wav_bins":  len(absorption), # x
    "time_bins": 150,             # y
    "max_time":  1.5,

    # "integrator": "acousticpath",
    "integrator": "prb_acoustic",
    # "integrator": "prb_reparam_acoustic",
    "max_depth": 50,
    "spp": 2**18,
}

fs = config["time_bins"] / config["max_time"]
time = np.linspace(0., config["max_time"], config["time_bins"], endpoint=False)

# config["max_depth"] = utils.estimate_max_depth(config["box_dim"], config["max_time"], 1.5)
print(f"max_depth = {config['max_depth']}")

scene_dict = utils.shoebox_scene(**config)
scene = mi.load_dict(scene_dict)

### Reference Histogram

In [ ]:
# hist_ref = mi.render(scene, seed=sess_seed)
# utils.plot_hist(hist_ref[:, :, 0], **config)

In [ ]:
rav_df = pd.read_pickle("../leo/notebooks/data/raven-box-25-12-7-hybrid.pkl")

idxs = []
for i in range(absorption.shape[0]):
    a, s = np.round(absorption[i], 1), np.round(scattering[i], 1)
    idxs.append(f"a{a}-s{s}")

rav = rav_df[idxs].to_numpy()

n   = int(1000 * config["max_time"] / config["time_bins"])
rav = np.sum(rav.reshape((rav.shape[0] // n, n, -1)), axis=1)
rav = rav[:config["time_bins"]]
rav = mi.TensorXf(np.dstack([rav, np.zeros_like(rav)]))

edc_rav = acoustic_torch.EDC(rav[:, :, 0], db=True, norm=True)

rav.shape

### Optimization Setup

In [ ]:
key = "acoustic_bsdf.absorption.values"
params = mi.traverse(scene)
display(params)
display(params[key])

In [ ]:
params_ref  = mi.Float(params[key])
params[key] = mi.Float([0.499, 0.501] * scattering_mdim.shape[0])

opt = mi.ad.Adam(lr=0.005)
opt[key] = params[key]
params.update(opt);

In [ ]:
def loss(a, b):
    a = acoustic_torch.C(a[:, :, 0], fs=fs)
    b = acoustic_torch.C(b[:, :, 0], fs=fs)
    # a = acoustic_torch.D(a[:, :, 0], fs=fs)
    # b = acoustic_torch.D(b[:, :, 0], fs=fs)
    # a = acoustic_torch.TS(mi.TensorXf(time), a[:, :, 0])
    # b = acoustic_torch.TS(mi.TensorXf(time), b[:, :, 0])

    # a     = acoustic_torch.EDC(a[:, :, 0], db=True, norm=True)
    # b     = acoustic_torch.EDC(b[:, :, 0], db=True, norm=True)
    # b     = edc_rav
    # a     = acoustic_torch.T(mi.TensorXf(time), a)
    # b     = acoustic_torch.T(mi.TensorXf(time), b)

    return utils.mse(a, b)

In [ ]:
hist = mi.render(scene, seed=sess_seed)
dr.enable_grad(hist)
l = loss(hist, rav)
dr.backward(l)
grad = dr.grad(hist)
utils.plot_hist(grad[:, :, 0], **config)

### Main Loop

In [ ]:
vals, losses = [], []
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
ax1.set_xlim(-1, 51)
ax1.set_ylim(-0.1, 1.1)
ax2.set_xlim(-1, 50)
ax2.set_ylim(-0.1, 1.1);
plt.close()

In [ ]:
iters  = 10
if iters > 1:
    n  = len(vals) + iters

# for i in trange(iters) if iters > 2 else range(iters):
for i in range(iters):
    hist = mi.render(scene, params, seed=sess_seed+i, seed_grad=sess_seed_g+i)
    l    = loss(hist[:], rav[:])
    dr.backward(l)

    if iters < 1:
        display(dr.grad(opt[key]))
        dr.set_grad(opt[key], 0)
    else:
        opt.step()
        opt[key] = dr.clamp(opt[key], 0.0, 1.0)
        params.update(opt)

        vals.append(params[key].numpy())
        losses.append(l.numpy()[0])

        ax1.clear()
        ax1.plot(vals)
        ax1.hlines([0.75, 0.1], 0, n, colors='k', linestyles='dotted')
        ax1.set_xlim(-n * 0.02, n * 1.02)
        ax1.set_ylim(-0.1, 1.1)
        ax2.clear()
        ax2.plot(losses)
        ax2.set_xlim(-1, n)
        fig.canvas.draw()

In [ ]:
if iters > 1:
    losses = np.array(losses)
    vals   = np.array(vals)
    data   = np.hstack([losses[:, None], vals])
    # np.save("/home/daniel/diff-absorption-edc-0_5s.npy", data)

In [ ]:
# a = [0.1,  0.75, 0.1 , 0.75,  0.1,  0.75]
# s = [0.5,  0.5 , 0.75, 0.75,  1. ,  1.  ]

# PRB - 5s fuer 10 iters, 1027 MiB GPU (unchanged to idle)
# without any dr.jit optims: 16s (ca. 3x so lange)
# [43.57299041748047, -26.953903198242188,
#  72.29619598388672, -45.544830322265625,
# 108.41382598876953, -67.37712097167969]

# acousticpath (AD) - >20min fuer 10 iters, 9930 MiB GPU
# [120.82818603515625, -79.3001937866211,
#  120.27494812011719, -74.47596740722656,
#  110.60392761230469, -68.03990936279297]

### Estimated (Forward) Gradients vs. Finite Differences

In [ ]:
a    = np.array([0.1, 0.1, 0.7, 0.7])
si   = np.array([1,     2,   1,   2], dtype=np.uint)

data = np.zeros((2, 2, a.shape[0], config["time_bins"]))

for t, sign in enumerate([1., -1.]):
    for k in range(a.shape[0]):
        # forward pass
        vals     = mi.Float([a[k]] * config["wav_bins"])
        opt[key] = vals
        params.update(opt)

        grad              = dr.zeros(mi.Float, config["wav_bins"])
        grad[int(si[k])]  = sign
        dr.set_grad(opt[key], grad)

        img               = mi.render(scene, params, seed=sess_seed, seed_grad=sess_seed_g)
        grad              = dr.forward_to(img * dr.rcp(config["spp"]))
        data[0, t, k]     = grad[:, int(si[k]), 0].numpy()

        # finite differences
        vals              = mi.Float([a[k]] * config["wav_bins"])
        opt[key]          = vals
        params.update(opt)
        img               = mi.render(scene, seed=sess_seed)

        vals              = mi.Float([a[k]] * config["wav_bins"])
        vals[int(si[k])]  = a[k] + sign * 1e-4
        opt[key]          = vals
        params.update(opt)
        finite_difference = mi.render(scene, seed=sess_seed) - img
        data[1, t, k]     = finite_difference[:, int(si[k]), 0].numpy()

# np.save("/home/daniel/diff-absorption-fd-vs-forward.npy", data)

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 8), sharex=True, sharey=False)

# ax1.plot(data[:, 0, [0, 1]].T.reshape((config["time_bins"], -1)))
# ax2.plot(data[:, 0, [2, 3]].T.reshape((config["time_bins"], -1)))
# ax3.plot(data[:, 1, [0, 1]].T.reshape((config["time_bins"], -1)))
# ax4.plot(data[:, 1, [2, 3]].T.reshape((config["time_bins"], -1)))
ax1.plot(data[0, 0].T.reshape((config["time_bins"], -1)))
ax2.plot(data[1, 0].T.reshape((config["time_bins"], -1)))
ax3.plot(data[0, 1].T.reshape((config["time_bins"], -1)))
ax4.plot(data[1, 1].T.reshape((config["time_bins"], -1)))

fig.show()

In [ ]:
utils.plot_hist(grad[:, :, 0], **config)
utils.plot_hist(finite_difference[:, :, 0], **config)